In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split



In [5]:
X,y = make_classification(n_samples=1000,n_features=10,n_informative=2,n_redundant=8,weights=[0.9,0.1],flip_y=0,random_state=42)


In [6]:
np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100]))

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [9]:
params = {
    "solver":"lbfgs",
    "max_iter":1000,
    "multi_class":"auto",
    "random_state":8888,
}

lr = LogisticRegression(**params)
lr.fit(X_train,y_train)

y_pred = lr.predict(X_test)

report_dict = classification_report(y_test,y_pred,output_dict=True)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [10]:
import mlflow
import mlflow.sklearn

In [12]:
mlflow.set_experiment("Third expirement")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'acc':report_dict['accuracy'],
        'recall_class_0':report_dict['1']['recall'],
        'recall_class_1':report_dict['1']['recall'],
        'f1_score_macro':report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr,"Logistic Regression")



2025/11/05 23:31:25 INFO mlflow.tracking.fluent: Experiment with name 'Third expirement' does not exist. Creating a new experiment.
2025/11/05 23:31:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/05 23:31:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run bustling-shrimp-519 at: http://127.0.0.1:5000/#/experiments/769311552355565496/runs/85a00ea1a4614ac3bed3732b51d7baae
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/769311552355565496


In [17]:
model_name = "Logistic Regression"
run_id=input('Enter runID:')
model_uri=f"runs:/{run_id}/{model_name}"

result =  mlflow.register_model(model_uri,model_name)

Registered model 'Logistic Regression' already exists. Creating a new version of this model...
2025/11/05 23:43:31 WARNING mlflow.tracking._model_registry.fluent: Run with id 85a00ea1a4614ac3bed3732b51d7baae has no artifacts at artifact path 'Logistic Regression', registering model based on models:/m-3a7583b495e1454b8e55354a7a9d6fd7 instead
2025/11/05 23:43:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression, version 2
Created version '2' of model 'Logistic Regression'.


In [15]:
model_uri=f"models:/Logistic Regression/1"
loaded_model= mlflow.sklearn.load_model(model_uri)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [ ]:
from mlflow.models import ModelVersion
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.transition_model_version_stage(
    name="RandomForestModel",
    version=result.version,
    stage="Production"
)

In [20]:
from mlflow.tracking import MlflowClient
model_name = "Logistic Regression"
model_version=1
dev_model_uri=f"models:/{model_name}/{model_version}"
prod_model='LogisticRegression_prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri,dst_name=prod_model)

Successfully registered model 'LogisticRegression_prod'.
Copied version '1' of model 'Logistic Regression' to version '1' of model 'LogisticRegression_prod'.


<ModelVersion: aliases=[], creation_timestamp=1762366574201, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1762366574201, metrics=None, model_id=None, name='LogisticRegression_prod', params=None, run_id='85a00ea1a4614ac3bed3732b51d7baae', run_link='', source='models:/Logistic Regression/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [21]:
import wandb

In [22]:
wandb.init(
    project="my_project_name",
    name="experiment_1",
    config={
        "learning_rate":0.001,
        "batch_size":32,
        "epochs":10
    }
)

config = wandb.config

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\DELL\_netrc
wandb: Currently logged in as: arya-22310290 (trials_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
